<a href="https://colab.research.google.com/github/Arrizky0911/Image-Processing-Recognition-Lab-Assignments/blob/main/assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab Assignment 2: Transforms & Filtering In Image Processing**

Name : Muhammad Arrizky Adhita Azizi

SID: 001202300019

Artificial Intelligence Class 2

**Background**

Medical images, such as MRI or CT scans, are crucial for diagnosing and monitoring
health conditions. However, these images often suffer from various issues:
- Noise: Artifacts from the scanning process can make it difficult to see important
details.
- Blurred details: Low resolution can hinder the identification of fine details.
- Large file sizes: Medical images are large, making storage and transfer difficult.

Thus, the need arises to enhance the quality of medical images through filtering and
transforming techniques while ensuring that no critical information is lost.

## **Setup Environment**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

%matplotlib inline

Execute the code below if you run it in colab (specify your folder images)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

FOLDERNAME = ''
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME

In [ ]:
original_image = cv2.imread('image.jpg', cv2.IMREAD_GRAYSCALE)

display(original_image)

In [ ]:
def display_images(images, num, titles = None, cmap=None):
  plt.figure(figsize=(15, 5))
  for i, img in enumerate(images):
    plt.subplot(1, num, i+1)
    plt.imshow(img, cmap)
    if titles is None:
      plt.title(f"Image {i+1}")
    else:
      plt.title(titles[i])
    plt.axis("off")
  plt.show()

## **Fourier Transform in Medical Imaging**

Fourier transforms is used to convert an image from the spatial domain to the frequency domain. This allows for the easy removal of high-frequency noise.

This process includes:
* Applying the 2D Fourier Transform to a noisy medical image.
* Filter out all the unwanted high-frequency components (i.e., noise) using low pass filter.
* Reconstructing the image back using the inverse Fourier transform to obtain a cleaner version of the image.

In [ ]:
# Apply 2D Fourier Transform
f_transform = np.fft.fft2(original_image)
f_shift = np.fft.fftshift(f_transform) # Shift zero frequency to center
magnitude_spectrum = 20 * np.log(np.abs(f_shift))

In [ ]:
# Create filters
rows, cols = original_image.shape
crow, ccol = rows // 2, cols // 2 # Center coordinates
low_pass = np.zeros((rows, cols), np.uint8)
cv2.circle(low_pass, (ccol, crow), 50, 1, -1) # Circle of radius 50

# Apply the filter
low_pass_filtered = f_shift * low_pass

In [ ]:
# Apply the Inverse Fourier Transform
f_iffshift = np.fft.ifftshift(low_pass_filtered)
reconstructed_image = np.abs(np.fft.ifft2(f_iffshift))

In [ ]:
fft_result_images = [original_image, magnitude_spectrum, reconstructed_image]
fft_result_titles = ["Original Image", "Magnitude Spectrum", "Reconstructed Image"]

display_images(
    fft_result_images,
    3,
    fft_result_titles,
    "grey")

## **Discrete Cosine Transform (DCT) in Image Compression**

Use DCT to compress large medical images without losing important diagnostic features.

This process includes:
- Applying the DCT to the image to transform it into frequency components.Which is gonna retain only the largest DCT coefficients, discarding smaller ones(which typically represent less important data).
- Reconstructing the image using the inverse DCT.

In [ ]:
# Step 1: # Ensure the image is in float32 format
image = np.float32(original_image)

# Step 2: Apply Discrete Cosine Transform (DCT)
dct_transform = cv2.dct(image)  # Apply 2D DCT
dct_visual = np.log(abs(dct_transform) + 1)  # Log scale for visualization

# Step 3: Retain the largest coefficients
compression_ratio = 0.1  # Retain 10% of the largest coefficients
flattened = abs(dct_transform).flatten()
threshold = np.sort(flattened)[::-1][int(len(flattened) * compression_ratio)]  # Calculate threshold
compressed_dct = np.where(abs(dct_transform) >= threshold, dct_transform, 0)  # Retain significant coefficients

# Step 4: Reconstruct the image using inverse DCT
reconstructed_image = cv2.idct(compressed_dct)  # Apply inverse DCT

In [ ]:
DCT_result_images = [original_image, dct_visual, reconstructed_image]
DCT_result_titles = ["Original Image", "DCT Spectrum", "Reconstructed Image"]
display_images(DCT_result_images, 3, DCT_result_titles, "grey")

In [ ]:
# Step 5: Report Compression
non_zero_original = np.count_nonzero(dct_transform)
non_zero_compressed = np.count_nonzero(compressed_dct)
compression_ratio_calculated =  non_zero_original / non_zero_compressed
print(f"Original Non-Zero Coefficients: {non_zero_original}")
print(f"Compressed Non-Zero Coefficients: {non_zero_compressed}")
print(f"Compression Ratio Achieved: {compression_ratio_calculated}")

## **Filtering in the Spatial Domain**

Apply spatial filters such as Sobel or Gaussian filters to enhance edges and fine details in the image.

This process includes:
- Applying a Gaussian filter to blur the image and reduce noise.
- Using a Sobel filter to highlight edges and fine details in the image.

In [ ]:
# Step 1: Apply Gaussian Filter
gaussian_blurred = cv2.GaussianBlur(original_image, (5,5), sigmaX=1)

# Step 2: Apply Sobel Filter
sobel_x = cv2.Sobel(gaussian_blurred, cv2.CV_64F, 1, 0, ksize=3)  # Sobel in the X direction
sobel_y = cv2.Sobel(gaussian_blurred, cv2.CV_64F, 0, 1, ksize=3)  # Sobel in the Y direction
sobel_combined = cv2.magnitude(sobel_x, sobel_y)  # Combine gradients

In [ ]:
filtering_result_images = [original_image, gaussian_blurred, sobel_x, sobel_y, sobel_combined]
filtering_result_titles = ["Original Image", "Gaussian Blurred", "Sobel X", "Sobel Y", "Sobel Combined"]

display_images(filtering_result_images, 5, filtering_result_titles, "grey")

## **Frequency Domain Filtering**

Use frequency domain filtering to improve image quality and assist with noise reduction and feature enhancement

This process includes:
- Applying a low-pass filter to the frequency domain representation to attenuate high-frequency components (typically associated with
noise).
- Applying a high-pass filter to highlight fine details and edges, enhancing the clarity of structural information within the image.
- Applying a band-pass filter to retain mid-frequency components that are important for certain features while removing both high and low-frequency noise.


In [ ]:
# Step 1: Apply 2D Fourier Transform
f_transform = np.fft.fft2(original_image)
f_transform_shifted = np.fft.fftshift(f_transform)  # Shift zero-frequency component to the center

# Get the magnitude spectrum for visualization
magnitude_spectrum = 20 * np.log(np.abs(f_transform_shifted))

# Step 2: Create filters
rows, cols = image.shape
crow, ccol = rows // 2, cols // 2 # Center coordinates

# 1. Low-pass filter
low_pass = np.zeros((rows, cols), np.uint8)
cv2.circle(low_pass, (ccol, crow), 50, 1, -1) # Circle of radius 50
low_pass_filtered = f_transform_shifted * low_pass

# 2. High-pass filter
high_pass = np.ones((rows, cols), np.uint8)
cv2.circle(high_pass, (ccol, crow), 50, 0, -1) # Circle of radius 50
high_pass_filtered = f_transform_shifted * high_pass

# 3. Band-pass filter
band_pass = np.zeros((rows, cols), np.uint8)
cv2.circle(band_pass, (ccol, crow), 80, 1, -1) # Outer circle
cv2.circle(band_pass, (ccol, crow), 30, 0, -1) # Inner circle
band_pass_filtered = f_transform_shifted * band_pass

# Step 3: Reconstruct Images using Inverse Fourier Transform
low_pass_image = np.abs(np.fft.ifft2(np.fft.ifftshift(low_pass_filtered)))
high_pass_image = np.abs(np.fft.ifft2(np.fft.ifftshift(high_pass_filtered)))
band_pass_image = np.abs(np.fft.ifft2(np.fft.ifftshift(band_pass_filtered)))

In [ ]:
frequency_filtering_result_images = [original_image, magnitude_spectrum, low_pass_image, high_pass_image, band_pass_image]
frequency_filtering_result_titles = ["Original Image", "Magnitude Spectrum", "Low-pass Filtered", "High-pass Filtered", "Band-pass Filtered"]

display_images(frequency_filtering_result_images, 5, frequency_filtering_result_titles, "grey")